In [1]:
import logging
import telebot
from config import TOKEN, LOG_FILE

import re
import os
import json
import nltk
import scipy
import pymystem3
import numpy as np
import pandas as pd

from nltk.stem.lancaster import LancasterStemmer
from nltk.tokenize import RegexpTokenizer

from nltk.tag import pos_tag
from nltk.stem.snowball import SnowballStemmer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from gensim.models import KeyedVectors
from gensim.models.word2vec import Word2Vec

from keras.models import load_model
import keras.backend as K

import tensorflow as tf

from itertools import chain
from tqdm import tqdm

C:\Anaconda3\envs\theano\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
class Backend(object):
    """
    class contets few machine learning text handle methods
    """
    
    def __init__(self, model_path, w2v_path, json_path):
        """
        initialize our class with model file, w2v file and json faq files 
        """
        self.model = load_model(model_path)
        self.w2v = Word2Vec.load(w2v_path)
        # self.faq = 
        self.tokenizer = RegexpTokenizer('[a-zA-Z0-9@]+')
        self.stemmer = LancasterStemmer()
        self.graph = tf.get_default_graph()
        
    def preproc_tweet(self, text):
        """
        function preprocesing tweet to good format
        """
        max_tweet_length = 20
        vector_size = 512
        
        matrix = np.zeros((1, max_tweet_length, vector_size), dtype=K.floatx())

        for t, token in enumerate(self.tokenizer.tokenize(text.lower())):
            if t >= max_tweet_length:
                break
            try:    
                matrix[0, t, :] = self.w2v[self.stemmer.stem(token)]
            except:
                pass
                
        return matrix
    
    def predict_sentiment(self, matrix):
        """
        fucntion returns sentiment
        argeuments:
            matrix - numpy array, matrix of w2v words ebeddings
            model - keras deep learning model
        """
        graph = self.graph
        global graph
        with graph.as_default():
            sentiment = self.model.predict(matrix)[0][1]
        
        return sentiment

<ipython-input-2-cf3fb7203897>:44: SyntaxWarning: name 'graph' is assigned to before global declaration
  global graph


In [3]:
# prepr = bk.predict_sentiment(bk.preproc_tweet('What a lonely day! And it\'s mine'))

In [4]:
bot = telebot.TeleBot(TOKEN)

In [5]:
# load some backend for bot
bk = Backend('../lecture1/weights-improvement-09-0.81.hdf5', '../lecture1/test.wv','')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
# Logging to file
fh = logging.FileHandler(LOG_FILE)
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
# Logging to console
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)

logger.addHandler(fh)
logger.addHandler(ch)

@bot.message_handler(commands=['start', 'help'])
def greetings(message):
    try:
        logger.info('Started chat # {}'.format(message.chat.id))
        bot.reply_to(message, 'Приветики, {} {}! Меня зовут {} (если коротко, то Сэнди), я к твоим услугам.\n'
                    'Вот что я могу:\n'
                    '/start - Начинаю чат и пишу тебе все, что умею (каждый раз, чтобы не забыл)\n'
                    '/help - Печатаю помощь (да, чтобы не забыл ;))\n'
                    '/sentiment [message] - предсказываю с помощью хрустального шара положительную (1) или отрицательную (0) окраску твита (только по-аглийски)\n'
                    '/answer [message] - Отвечаю по моему справочнику FAQ (надеюсь, что это не что-то неприличное) на твое сообщение'.format(message.chat.first_name,
                                                                        message.chat.last_name, bot.get_me().username))
    except:
        bot.send_message(message.chat.id, 'Извини, я ошиблась. С кем не бывает?!')
        
@bot.message_handler(commands=['sentiment'])
def predict_sentiment(message):
    try:
        text = ' '.join(message.text.split(' ')[1:]) # some idiot bicycle to delete command from text
        result = bk.predict_sentiment(bk.preproc_tweet(text))
        bot.send_message(message.chat.id, result)
        logger.info('Predicted sentiment {} for tweet {}'.format(result, text))
    except:
        bot.send_message(message.chat.id, 'Извини, я ошиблась. С кем не бывает?!')
        
@bot.message_handler(commands=['answer'])
def answer(message):
    bot.send_message(message.chat.id, 'Ответ мой: 42')
    
@bot.message_handler(content_types=['text'])
def small_talk(message):
    bot.send_message(message.chat.id, "Я бы с тобой поговорила, да вот сказать нечего :(\n Введи-ка лучше команду.")

In [6]:
if __name__ == '__main__':
    bot.polling(none_stop=True)

2017-12-03 13:25:35,663 - __main__ - INFO - Started chat # 74141678
2017-12-03 13:27:08,034 - __main__ - INFO - Predicted sentiment 0.8989554047584534 for tweet We will rise again
2017-12-03 13:27:24,596 - __main__ - INFO - Predicted sentiment 0.009651520289480686 for tweet I am so sick


In [ ]:
def main():
    # Enable logging


    # @bot.message_handler(content_types=["text"])
    # def repeat_all_messages(message): # Название функции не играет никакой роли, в принципе
    #     bot.send_message(message.chat.id, message.text)

    @bot.message_handler(commands=['start', 'help'])
    def greetings(message):
        try:
            bot.reply_to(message, 'Greetings, {} {}! My name is {}, at your service.\n'
                        'I currently support the following commands:\n'
                        '/start - begins our chat and prints this message\n'
                        '/help - prints this message\n'
                        '/sentiment [message] - predicts the sentiment of the message (only english)\n
                        /answer [message] - answers to user message'.format(message.chat.first_name,
                                                                            message.chat.last_name, bot.get_me().username))
        except:
            
    @bot.message_handler(commands)

In [ ]:
if __name__ == '__main__':
    main()
    bot.polling(none_stop=True)

In [ ]:
# Enable logging
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
# Logging to file
fh = logging.FileHandler(LOG_FILE)
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
# Logging to console
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
ch.setFormatter(formatter)

logger.addHandler(fh)
logger.addHandler(ch)


class Bot:
    def __init__(self):

        self.updater = Updater(TOKEN)
        self.dsp = self.updater.dispatcher
        
        self.text = 'TEXT'

        # register handler functions which define how the bot reacts to events
        self.dsp.add_handler(CommandHandler("start", get_help))
        self.dsp.add_handler(CommandHandler("help", get_help))
        self.dsp.add_handler(CommandHandler("sentiment", get_sentiment, pass_chat_data = True))
        self.dsp.add_handler(CommandHandler("answer", get_answer))
        self.dsp.add_handler(MessageHandler(Filters.text, echo))
        self.dsp.add_error_handler(error)

        logger.info('Im alive!')
        
#     def load_model(self, w2v, model):
#         # load w2v and 
    
#     def preproc_

    def power_on(self):
        # start the Bot
        self.updater.start_polling()
        self.updater.idle()

# define command handlers. These usually take the two arguments: bot and
# update. Error handlers also receive the raised TelegramError object in error.


def echo(bot, update):
    logger.info('echo recieved message: {}'.format(update.message.text))
    bot.sendMessage(update.message.chat_id, text=update.message.text)


def error(bot, update, error):
    # all uncaught telegram-related exceptions will be rerouted here
    logger.error('Update "%s" caused error "%s"' % (update, error))


def get_help(bot, update):
    logger.info('get_help recieved message: {}'.format(update.message.text))
    help_msg = ('Greetings, {} {}! Name is {}, at your service.\n'
                'I currently support the following commands:\n'
                '/start - begins our chat and prints this message\n'
                '/help - prints this message\n'
                '/sentiment [message] - predicts the sentiment of the message').format(
        update.message.from_user.first_name, update.message.from_user.last_name, bot.name)
    bot.sendMessage(update.message.chat_id, text=help_msg)
    
# def load_model(bot, update, chat_data):
    

def get_sentiment(bot, update, chat_data):
    logger.info('get_sentiment recieved message: {}'.format(update.message.text))
    try:
        # get message text without the command '/sentiment'
        usr_msg = update.message.text.split(' ', maxsplit=1)[1]
        msg_sentiment = 0.5
        '''
        Now determine the sentiment of usr_msg.
        This should return a real number in [0,1].
        Your code goes here.
        '''
        
        msg_sentiment = bot.text
        
        bot.sendMessage(update.message.chat_id, text=msg_sentiment)
    except IndexError:
        bot.sendMessage(update.message.chat_id, text='Write your message after the command')
    except Exception as e:
        logger.error(e)
        
def get_answer(bot, update):
    logger.info('get_answer recieved message: {}'.format(update.message.text))
    try:
        # get message text without the command 
        usr_msg = update.message.text.split(' ', maxsplit=1)[1]
        # your code goes here
        '''
        Now determine the sentiment of usr_msg.
        This should return a real number in [0,1].
        Your code goes here.
        '''
        bot.sendMessage(update.message.chat_id, text=answer)
    except IndexError:
        bot.sendMessage(update.message.chat_id, text='Write your message after the command')
    except Exception as e:
        logger.error(e)

my_bot = Bot()
my_bot.power_on()